# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np
from IPython.display import display, HTML # for pretty printing

url = "..\..\\nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU", 'BPXDI1']
da = da.dropna(subset = vars)

da['RIAGENDRx']= da['RIAGENDR'].replace({1:'Male', 2:'Female'})
# This is the grouping variable
da["group"] = 10*da['SDMVSTRA'] + da['SDMVPSU']

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [10]:
# enter your code here
model_DI = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result_DI = model_DI.fit()
print('BPXDI1',result_DI.cov_struct.summary())

model_SY = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result_SY = model_SY.fit()
print('BPXSY1', result_SY.cov_struct.summary())

BPXDI1 The correlation between two observations in the same cluster is 0.031
BPXSY1 The correlation between two observations in the same cluster is 0.030


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

The ICC for BPXDI1 is .031 which is exactly the same for BPXSY1

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [11]:
# enter your code here
model_MV = sm.GEE.from_formula("BPXDI1 ~ BMXBMI + RIDAGEYR + RIAGENDRx", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result_MV = model_MV.fit()
print('BPXDI1 ~ BMXBMI + RIDAGEYR + C(RIAGENDRx)',result_MV.cov_struct.summary())
result_MV.summary()

BPXDI1 ~ BMXBMI + RIDAGEYR + C(RIAGENDRx) The correlation between two observations in the same cluster is 0.030


<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                      BPXDI1   No. Observations:                 5102
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 106
                      Estimating Equations   Max. cluster size:                 226
Family:                           Gaussian   Mean cluster size:               170.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Sun, 08 Aug 2021   Scale:                         158.611
Covariance type:                    robust   Time:                         18:53:32
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            65.3013      1.235     52.856      0.000      62.880      67.723
RIAGENDRx[T.Male]     2.7422      0.402      6.828      0.000       1.955       3.529
BMXBMI                0.1839      0.030      6.218      0.000       0.126       0.242
RIDAGEYR             -0.0409      0.014     -2.845      0.004      -0.069      -0.013
==============================================================================
Skew:                         -0.6865   Kurtosis:                       3.9310
Centered skew:                -0.6664   Centered kurtosis:              3.9656
==============================================================================
"""

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC is 0.030. The ICC was reduced by .001 and is possible that the other covariates or IV explained part of the ICC. So when controlling for those covariates the ICC value was reduced although negligible.

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [12]:
# enter your code here
df = da[da['RIAGENDRx'] == 'Female']
dm = da[da['RIAGENDRx'] == 'Male']

model_M = sm.GEE.from_formula("BPXDI1 ~ BMXBMI + RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=dm)
result_M = model_M.fit()
print('Male:',result_M.cov_struct.summary())
display(result_M.summary())

model_F = sm.GEE.from_formula("BPXDI1 ~ BMXBMI + RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=df)
result_F = model_F.fit()
print('\nMale:',result_F.cov_struct.summary())
display(result_F.summary())


Male: The correlation between two observations in the same cluster is 0.035


<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                      BPXDI1   No. Observations:                 2462
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  38
                      Estimating Equations   Max. cluster size:                 111
Family:                           Gaussian   Mean cluster size:                82.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Sun, 08 Aug 2021   Scale:                         173.641
Covariance type:                    robust   Time:                         18:53:32
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.8949      1.751     38.210      0.000      63.464      70.326
BMXBMI         0.2652      0.055      4.846      0.000       0.158       0.372
RIDAGEYR      -0.0661      0.017     -3.947      0.000      -0.099      -0.033
==============================================================================
Skew:                         -0.7533   Kurtosis:                       4.0786
Centered skew:                -0.7013   Centered kurtosis:              4.0070
==============================================================================
"""


Male: The correlation between two observations in the same cluster is 0.029


<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                      BPXDI1   No. Observations:                 2640
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  59
                      Estimating Equations   Max. cluster size:                 125
Family:                           Gaussian   Mean cluster size:                88.0
Dependence structure:         Exchangeable   Num. iterations:                     8
Date:                     Sun, 08 Aug 2021   Scale:                         143.982
Covariance type:                    robust   Time:                         18:53:32
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     65.7505      1.348     48.778      0.000      63.109      68.392
BMXBMI         0.1324      0.030      4.374      0.000       0.073       0.192
RIDAGEYR      -0.0192      0.019     -0.998      0.318      -0.057       0.019
==============================================================================
Skew:                         -0.6088   Kurtosis:                       3.6004
Centered skew:                -0.5958   Centered kurtosis:              3.6229
==============================================================================
"""

__Q3a.__ What do you learn by comparing these two fitted models?

The ICC is indeed controlled for the gender. When controlling by gender by separating in groups the correlation drops in the female case and goes up in the male case.

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [22]:
# enter your code here
model_RI = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx",
           groups="group", re_formula = '1', data=da)
result_RI = model_RI.fit()
print(result_RI.summary())

model_ARI = sm.MixedLM.from_formula("BPXDI1 ~ RIAGENDRx",
           groups="RIDAGEYR", re_formula = '1', data=da)
result_ARI = model_ARI.fit()
print(result_ARI.summary())

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  BPXDI1     
No. Observations:   5102     Method:              REML       
No. Groups:         30       Scale:               155.7897   
Min. group size:    106      Log-Likelihood:      -20145.7435
Max. group size:    226      Converged:           Yes        
Mean group size:    170.1                                    
-------------------------------------------------------------
                  Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept         70.763    0.661 107.026 0.000 69.467 72.059
RIAGENDRx[T.Male]  2.545    0.351   7.258 0.000  1.857  3.232
RIDAGEYR          -0.040    0.010  -3.997 0.000 -0.060 -0.021
group Var          4.082    0.105                            

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  BPXDI1     
No. Observations:   5102     Me

__Q4a.__ How would you describe the strength of the clustering in this analysis?

Looking at the p value of the parameters calculated for the fixed effects, one would think that they are significant. However the log-likelihood is quite low compared to the model that incorporates age as the higher grouping level.

__Q4b:__ Include a random intercept for age, and describe your findings.

In this case the p value of the paramters calculated for fixed effects (only gender) is significant. And the variance between groups is quite high. In addition to that the log-likelihood is better than the previous model